# Are you interested in learning how to handle multiple RNA-seq dataset ??

   ### Here is a easy way to learn to build custom pipeline for rna-seq data processing

   # Try it!
   
   
   
   
     Here, the goal is to help the beginners to start from scratch and master in designing custom pipeline for
     Next-gen sequencing data processing..
   
##### NOTE: This coding exercise is for the beginners who want to learn the basics of pipeline designing for handing multiple RNA-seq datasets in a single batch. Ofcourse there are other highly advanced pipelines available such as Snakemake (https://snakemake.readthedocs.io/en/stable/ ) and Galaxy (https://usegalaxy.org/) which have a broad application, not just limited to RNA-seq.

## How to use ?

### Requirements
        Make sure the module(s) are installed in your unix environment
        Also, the python packages are compatible with your python version
        It is a good idea to do thorough quality check of your raw RNA seq files
        using tools such as FASTQC and if required preprocess the reads using read
        read editor tools such as Trimmomatic,FASTx toolkits etc.
        
#### The code is written in python. All you need to do is to run the python script in bash as shown below

        Define the parameters as per your choice
        Use the module(s) required for data processing
        Define the directory of raw fastq file(s)
        Define the output directory    
        
$python Pipeline_Fastq-to-Quant.py -h

## First lets talk a bit more about this pipeline...

### Work flow of the current pipeline is as follows:

#### Wait... are you trying for a short cut ?

👀

👀

👀

👀

👀

👀

👀

👀

👀


🤷‍

## Why dont you open the python script ?

In [ ]:
# Import packages for python
import os
import argparse


'''
You can always customize this code to run your job(s) in High Performance Computing system using PBS or
in your local machine. 

Technically this code is flexible with any cluster. 
Just change the JobTemplate!
'''

JobTemplate = """
#!/bin/bash
#PBS -l nodes=%d:ppn=%d 
#PBS -l walltime=%s 
#PBS -l gres=ccm
#PBS -N %s"""

#ARGPARSE is a beautiful and efficient python package that help you to customise the parameters as per your choice 

def main():
    parser = argparse.ArgumentParser(
                    description='Custom pipeline to generate script(s) for RNA-seq data processing and quant',
                    formatter_class=argparse.ArgumentDefaultsHelpFormatter
                                    )
    parser.add_argument('-file', metavar='fq', type = str, nargs='*', help='input fast(q|a) file(s), option:path/*.fastq')
    parser.add_argument('-modules', metavar='module', nargs='*', help='module(s) required for data processing')
    parser.add_argument('-n', type = int, default = 1, help='number of nodes')
    parser.add_argument('-p', type = int, default = 16, help='number of processors')
    parser.add_argument('-w', type = str, default = '04:00:00', help='estimated wall time for process')
    parser.add_argument('-index', type = str, default = 'Reference/mouse/index/mm10*', help='<path>/reference index file(s)')
    parser.add_argument('-gtf', type = str, default = 'Reference/mouse/mm10.gtf', help='<path>/reference annotation file')
    parser.add_argument('-odir', type = str, default = '', help='output directory path')
    args = parser.parse_args()

    '''
    You know, what is happening here. 
    We are just defining the parameters based on our arguments
    Ofcourse this is not the end, but it will give you an idea on how
    to add parameters for your pipeline/ tool.
    
    
    One step smarter now... 
    
    Lets move on..
    
    Just follow the 3 steps
    '''
    
    
    
    for x in args.file:
        x_name = x.split('/')[-1].split('.')[0]
        x_dir = os.path.dirname(x)
        with open(x + '.sh', 'w') as f:
            f.write(JobTemplate %(args.n,args.p,args.w,x_name)+'\n\n')       #Did you notice I am calling JobTemplate here!
                                                                             #os.path.dirname(x) for calling fastq directory 
            for m in args.modules:
                f.write('module load '+m+'\n')
            f.write('\ncd '+x_dir+'\n\n')
            
            
            '''
            STEP 1: ALIGNER
            User can add multiple aligner here and make use of it by tweaking in the below code as per the user defined
            paramenters in tool guidelines:
            
            if 'aligner' in str(args.modules):
                f.write('ccmrun aligner -p '+str(args.p)+ ' -q -x '+str(args.index)+ ' -u '+x+ ' -S ' + args.odir+x_name+
                '.sam'+'\n')  
            ## Customise parameters as per manual instructions
            '''
            
            
            if 'hisat' in str(args.modules):
                f.write('ccmrun hisat -p '+str(args.p)+ ' -q -x '+str(args.index)+ ' -u '+x+ ' -S ' + args.odir+x_name+
                        '.sam'+'\n')
            
            '''
            STEP 2: POST-PROCESSING
            SAMTools is used for post processing of aligned reads. This tool can be customised as per user choice.
            In case, the aligner is providing the processed reads, samtool module can be ignored. 
            Example - TopHat!            
            '''      
                       
            if 'samtools' in str(args.modules):
                f.write('ccmrun samtools view -bS '+args.odir+x_name+'.sam > '+ args.odir+x_name+'.bam'+'\n')
                f.write('ccmrun samtools sort '+args.odir+x_name+'.bam '+ args.odir+x_name+'.sorted'+'\n')
                f.write('ccmrun samtools index '+args.odir+x_name+'.sorted.bam'+'\n')
            
                        
            '''
            STEP 3: QUANTIFIER
            User can add multiple quantification tool and make use of it by tweaking in the below code as per the user
            defined paramenters in tool guidelines:
            '''
            
            
            if 'stringtie' in str(args.modules):
                f.write('ccmrun stringtie '+args.odir+x_name+'.sorted.bam -p '+str(args.p)+ ' -G '+str(args.gtf)+
                        ' -e -o '+args.odir+x_name+'.gtf'+' -A '+args.odir+x_name+'.tab'+'\n')

    return
if __name__ == "__main__":
    main() 

### Lets take a test run in bash (assuming all the essential requirements are fulfilled)

     RUN THIS >>>

     $python Pipeline_Fastq-to-Quant.py -file FASTQ/*.py -modules ccm samtools/1.9 stringtie/1.2.3 hisat/3.2.1 -w 08:00:00             -odir FASTQ/DATA/ -n 4 -t 1
     
#### Did you notice, what is happening here?

     We are calling python script with custom parameters such as -w (walltime), -n (number of nodes), 
     -t (number of processors) , -odir (giving the path for output) and required modules etc to generate
     scripts for all the fastq files.
     
     
#### Here is the *output* *script* you will be getting as a Filename.sh:
     
    #!/bin/bash
    #PBS -l nodes=4:ppn=12 
    #PBS -l walltime=08:00:00 
    #PBS -l gres=ccm
    #PBS -N fq1

    module load ccm
    module load samtools/1.9
    module load stringtie/1.2.3
    module load hisat/3.2.1

    cd FASTQ

    ccmrun hisat -p 12 -q -x reference/mouse/index/mm10* -u FASTQ/fq1.py -S FASTQ/DATA/fq1.sam
    ccmrun samtools view -bS FASTQ/DATA/fq1.sam > FASTQ/DATA/fq1.bam
    ccmrun samtools sort FASTQ/DATA/fq1.bam FASTQ/DATA/fq1.sorted
    ccmrun samtools index FASTQ/DATA/fq1.sorted.bam
    ccmrun stringtie FASTQ/DATA/fq1.sorted.bam -p 12 -G reference/mouse/mm10.gtf -e -o FASTQ/DATA/fq1.gtf -A FASTQ/DATA/fq1.tab
    
    
    
### Can you please look back at the code to revive all the 3 STEPS in the output   
     
     
     

### What next...

Once you run the python program with customized parameters, you will get the script for each Fastq (Raw RNA seq) file.

In Portable Batch System (PBS), you have to run the job as follows:

            $qsub Filename.sh

However, if you are running multiple scripts, you can que the jobs in the cluster by using following command while in the directory where codes are stored.

            $for f in $(find -name '*.sh'); do qsub $f; done
            

### Please share your feedback as comments. If you find it challenging, lets connect we will resolve together


### Please follow our page and hashtag to learn about some interesting assignments in coming days..

# We hope you enjoyed this tutorial :)  

## Happy coding...